### Model Training

In [1]:
import csv
import pandas as pd

In [2]:
try:
    import models
except ImportError:
    print('Please upload models.py to you current directory')
    raise

2024-04-21 21:17:09.546194: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 21:17:09.595539: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 21:17:10.356462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_pickle("./data/cve")

In [4]:
train = df[:120000]
test = df[120000:]

In [5]:
from models import BERTmodel

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100 80GB PCIe


In [7]:
metric = 'attack_complexity'

In [10]:
model = BERTmodel(metric, train, test)

In [11]:
model.train(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


======== Epoch 1 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:33:06.
  Batch 4,000  of  13,500.    Elapsed: 1:06:37.
  Batch 6,000  of  13,500.    Elapsed: 1:40:08.
  Batch 8,000  of  13,500.    Elapsed: 2:13:40.
  Batch 10,000  of  13,500.    Elapsed: 2:47:12.
  Batch 12,000  of  13,500.    Elapsed: 3:19:44.

  Average training loss: 0.18
  Training epoch took: 3:39:20

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.16
  Validation took: 0:04:56

======== Epoch 2 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:20:06.
  Batch 4,000  of  13,500.    Elapsed: 0:40:13.
  Batch 6,000  of  13,500.    Elapsed: 1:00:20.
  Batch 8,000  of  13,500.    Elapsed: 1:20:27.
  Batch 10,000  of  13,500.    Elapsed: 1:40:36.
  Batch 12,000  of  13,500.    Elapsed: 2:00:45.

  Average training loss: 0.15
  Training epoch took: 2:15:48

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.18
  Validation took: 0:04:35

Training complete!
Total tr

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.18,0.16,0.96,3:39:20,0:04:56
2,0.15,0.18,0.96,2:15:48,0:04:35


In [12]:
model.test()

0.9533875338753387

In [13]:
print(model.getClassificationStats())

{'accuracy': 0.9533875338753387, 'precision': 0.9533875338753387, 'recall': 0.9533875338753387, 'f1': 0.9533875338753387, 'mcc': 0.45633988088875127, 'cm': array([[29379,   140],
       [ 1322,   524]])}


In [14]:
model.saveModel()
model.savePredictions()

### Push model to huggingface hub

In [15]:
from huggingface_hub import notebook_login

In [16]:
notebook_login()

In [17]:
from transformers import BertForSequenceClassification

In [18]:
model = BertForSequenceClassification.from_pretrained(f"./{metric}_model/")

In [19]:
model.push_to_hub(f"Vijaybhk/{metric.title()}-BERT")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vijaybhk/Attack_Complexity-BERT/commit/efae192fb03a135a317e5c5a16bac859d26e606c', commit_message='Upload BertForSequenceClassification', commit_description='', oid='efae192fb03a135a317e5c5a16bac859d26e606c', pr_url=None, pr_revision=None, pr_num=None)